In [17]:
# Cell 0: Configure matplotlib
%matplotlib inline
# Alternative: Use the notebook backend
%matplotlib notebook

# Project Title: Real-Time 2D Object Detection with YOLOv8 on Pascal VOC Dataset

## Introduction

Object detection is a fundamental task in computer vision with wide-ranging applications, from autonomous driving to surveillance and medical imaging. This project aims to implement and evaluate a state-of-the-art deep learning model, YOLOv8, for 2D object detection. We will utilize the well-established Pascal VOC 2007 dataset to train and test our model, focusing on achieving a balance between detection accuracy and inference speed. This notebook (and accompanying Medium post) will walk through the entire pipeline, from data selection and preprocessing to model training, evaluation, and result interpretation.

## Project Objectives and Goals

The primary objectives of this project are:
1.  To understand and implement a modern object detection pipeline using a deep learning approach.
2.  To gain hands-on experience with the YOLOv8 architecture, a popular and efficient object detection model.
3.  To effectively preprocess the Pascal VOC 2007 dataset for training with YOLOv8.
4.  To train the YOLOv8 model on the prepared dataset and evaluate its performance using standard object detection metrics.
5.  To analyze the results and discuss the model's strengths, weaknesses, and potential areas for improvement.

The goal is to develop a functional object detection system capable of identifying and localizing multiple objects within an image with reasonable accuracy.

## Data Selection: Pascal VOC 2007

For this project, we have selected the **Pascal Visual Object Classes (VOC) 2007 dataset**.

**Source:** The dataset is publicly available and widely used in the computer vision community. It can typically be downloaded from the official Pascal VOC website or various academic mirrors.
*   Official Site (historical): [http://host.robots.ox.ac.uk/pascal/VOC/voc2007/](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/)

**Suitability:**
*   **Benchmark Dataset:** VOC 2007 is a standard benchmark for object detection tasks, making it suitable for comparing our model's performance against established results.
*   **Variety of Objects:** It contains 20 object classes, offering a diverse set of categories for detection (e.g., person, car, cat, dog, chair, bottle).
*   **Realistic Scenarios:** The images depict realistic scenes with objects in various contexts, scales, and levels of occlusion.
*   **Annotation Availability:** It provides detailed annotations, including bounding boxes and class labels for objects in the images, which are crucial for supervised training.
*   **Manageable Size:** While comprehensive, its size is manageable for training on typical modern hardware within a reasonable timeframe.

Our current setup uses the VOC 2007 data located within the `datasets/VOCdevkit/VOC2007/` directory in this project's workspace.

## Data Description

The Pascal VOC 2007 dataset consists of images and their corresponding annotations.

*   **Data Format:**
    *   **Images:** Primarily JPEG (`.jpg`) files.
    *   **Annotations:** Originally provided in XML format (`.xml`), one file per image. Each XML file contains information about the image (filename, size) and details for each annotated object (class label, bounding box coordinates: xmin, ymin, xmax, ymax).

*   **Dataset Size (VOC 2007 Main Competition):**
    *   **Total Images:** Approximately 9,963 images.
    *   **Training/Validation Images:** 5,011 images.
    *   **Test Images:** 4,952 images.
    *   **Object Classes:** 20 distinct classes.

*   **Attributes (Object Classes):**
    The 20 object classes are:
    `aeroplane`, `bicycle`, `bird`, `boat`, `bottle`, `bus`, `car`, `cat`, `chair`, `cow`, `diningtable`, `dog`, `horse`, `motorbike`, `person`, `pottedplant`, `sheep`, `sofa`, `train`, `tvmonitor`.
    *(Note: The `background` class is implicit and handled during training).*

*   **Inherent Challenges or Limitations:**
    *   **Class Imbalance:** Some object classes appear more frequently than others, which can bias the model.
    *   **Object Scale Variation:** Objects appear at various sizes, from very small to large, posing a challenge for detection models.
    *   **Occlusion:** Objects can be partially or heavily occluded by other objects.
    *   **Intra-class Variation:** Objects within the same class can have significant visual differences.
    *   **Clutter and Complex Scenes:** Images can be cluttered with multiple objects and complex backgrounds.
    *   **Annotation Quality:** While generally good, there might be minor inconsistencies or inaccuracies in annotations.
    *   **Dataset Size:** While used as a general benchmark, this dataset is rather small for deep learning tasks and will likely require data augmentation and/or transfer learning


**Alignment with Project Objectives:**
The VOC 2007 dataset directly aligns with our project objectives as it provides the necessary annotated image data to train and evaluate a supervised object detection model like YOLOv8. The variety of classes and challenging scenarios allow for a robust assessment of the model's capabilities.

## 5. Data Preprocessing (5%)

As outlined previously, the Pascal VOC 2007 dataset requires conversion to be compatible with YOLOv8. Our `train_yolov8_model.py` script (and the functions we'll show below, extracted from it) handles this. The key steps are:

1.  **Annotation Conversion & Directory Structuring**: Handled by `convert_voc_to_yolo`.
2.  **Dataset YAML Configuration**: Handled by `create_dataset_yaml`.

Let's look at the Python functions responsible for these steps.

In [1]:
# Cell 1: Imports and Setup
import os
import yaml
import torch
import xml.etree.ElementTree as ET
import shutil
from tqdm import tqdm
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print(f"Using device: {device}")


Using GPU: NVIDIA GeForce RTX 4090 Laptop GPU


In [2]:
# Cell 2: Convert VOC to YOLO Format

# VOC class names (matching your VOC dataset)
VOC_CLASSES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

def convert_voc_to_yolo(voc_path, output_path):
    """
    Convert VOC format dataset to YOLO format
    
    Args:
        voc_path: Path to the VOC dataset (e.g., datasets/VOCdevkit/VOC2007)
        output_path: Path to save the converted YOLO format dataset
    
    Returns:
        Path to the converted dataset
    """
    print(f"Converting VOC dataset from {voc_path} to YOLO format at {output_path}")
    
    voc_path = Path(voc_path).absolute()
    output_path = Path(output_path).absolute()
    
    # Create output directories
    train_img_dir = output_path / 'train' / 'images'
    train_label_dir = output_path / 'train' / 'labels'
    val_img_dir = output_path / 'val' / 'images'
    val_label_dir = output_path / 'val' / 'labels'
    
    # Clear existing data if any
    if output_path.exists():
        print(f"Output directory {output_path} already exists. Removing existing data...")
        shutil.rmtree(output_path, ignore_errors=True)
    
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)
    
    # Get train/val split from ImageSets
    train_ids = []
    val_ids = []
    
    # Read trainval split
    with open(voc_path / 'ImageSets' / 'Main' / 'trainval.txt', 'r') as f:
        trainval_ids = [line.strip() for line in f if line.strip()]
    
    # If test.txt exists, use it for validation, otherwise split trainval
    test_file = voc_path / 'ImageSets' / 'Main' / 'test.txt'
    if test_file.exists():
        with open(test_file, 'r') as f:
            val_ids = [line.strip() for line in f if line.strip()]
        train_ids = trainval_ids
    else:
        # Use 80% for training, 20% for validation
        split_idx = int(len(trainval_ids) * 0.8)
        train_ids = trainval_ids[:split_idx]
        val_ids = trainval_ids[split_idx:]
    
    print(f"Found {len(train_ids)} training images and {len(val_ids)} validation images")
    
    # Convert annotations and copy images
    def process_set(image_ids, img_output_dir, label_output_dir):
        for img_id in tqdm(image_ids, desc="Converting annotations"):
            # Copy image
            src_img = voc_path / 'JPEGImages' / f"{img_id}.jpg"
            if not src_img.exists():  # Try png if jpg doesn't exist
                src_img = voc_path / 'JPEGImages' / f"{img_id}.png"
                if not src_img.exists():
                    print(f"Warning: Image {img_id} not found")
                    continue
            
            dst_img = img_output_dir / f"{img_id}.jpg"
            shutil.copy(src_img, dst_img)
            
            # Convert annotation
            anno_path = voc_path / 'Annotations' / f"{img_id}.xml"
            if not anno_path.exists():
                print(f"Warning: Annotation for {img_id} not found")
                continue
            
            # Parse XML
            tree = ET.parse(anno_path)
            root = tree.getroot()
            
            # Get image size
            size = root.find('size')
            img_width = int(size.find('width').text)
            img_height = int(size.find('height').text)
            
            # Process objects
            yolo_lines = []
            for obj in root.findall('object'):
                # Get class name
                class_name = obj.find('name').text
                if class_name in VOC_CLASSES:
                    class_id = VOC_CLASSES.index(class_name) - 1  # Subtract 1 to remove background class
                    if class_id < 0:  # Skip background
                        continue
                else:
                    print(f"Warning: Unknown class {class_name} in {img_id}")
                    continue
                
                # Get bounding box
                bbox = obj.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)
                
                # Convert to YOLO format: class_id x_center y_center width height
                x_center = (xmin + xmax) / (2 * img_width)
                y_center = (ymin + ymax) / (2 * img_height)
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height
                
                # Add to YOLO format lines
                yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}")
            
            # Write YOLO format annotation
            with open(label_output_dir / f"{img_id}.txt", 'w') as f:
                f.write('\n'.join(yolo_lines))
    
    # Process train and validation sets
    process_set(train_ids, train_img_dir, train_label_dir)
    process_set(val_ids, val_img_dir, val_label_dir)
    
    # Create class.txt file
    with open(output_path / "class.txt", 'w') as f:
        # Skip background class (index 0)
        f.write('\n'.join(VOC_CLASSES[1:]))
    
    print("Conversion completed!")
    return str(output_path)

In [3]:
# Cell 3: Create Dataset YAML Config
def create_dataset_yaml(dataset_path, name="voc_dataset"):
    """
    Create a YAML file for the dataset configuration
    """
    dataset_path = Path(dataset_path).absolute()
    yaml_path = dataset_path / f"{name}.yaml"
    
    # Create YAML content with absolute paths
    data = {
        'path': str(dataset_path),
        'train': str(dataset_path / 'train' / 'images'),
        'val': str(dataset_path / 'val' / 'images'),
        'test': str(dataset_path / 'val' / 'images'),  # Use val as test set too
        'names': {}
    }
    
    # Check if class.txt exists to define class names
    class_file = dataset_path / "class.txt"
    if class_file.exists():
        with open(class_file, 'r') as f:
            class_names = f.read().strip().split('\n')
            data['names'] = {i: name for i, name in enumerate(class_names)}
    
    # Write YAML file
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    
    print(f"Created dataset config at {yaml_path}")
    return str(yaml_path)

## 6. Model Implementation (10%)

For this project, we selected the **YOLOv8 (You Only Look Once version 8)** model, a state-of-the-art, real-time object detection system developed by Ultralytics.

### 6.1. Model Choice: YOLOv8

**Rationale:**
*   **Performance:** YOLOv8 offers an excellent balance of speed and accuracy, making it suitable for a wide range of applications, including those requiring real-time processing.
*   **Ease of Use:** The Ultralytics framework provides a user-friendly Python API for training, validation, and inference with YOLOv8 models.
*   **Scalability:** It comes in various sizes (nano `n`, small `s`, medium `m`, large `l`, extra-large `x`), allowing users to choose a model that best fits their computational resources and performance requirements. For this project, we primarily focus on `yolov8n.pt` (nano version) for faster training and iteration, demonstrating the pipeline.
*   **Active Development:** YOLOv8 is actively maintained and updated with the latest advancements in object detection.

### 6.2. Architecture Overview (Brief)

YOLOv8 builds upon the successes of its predecessors. Key architectural features typically include:
*   **Backbone:** A powerful feature extractor (e.g., based on CSPDarknet principles).
*   **Neck:** A feature aggregation network (like PANet or BiFPN) that combines features from different scales to improve detection of objects of various sizes.
*   **Head (Detection Head):** Responsible for predicting bounding boxes, class probabilities, and objectness scores. YOLOv8 uses an anchor-free, decoupled head, which can lead to faster NMS and improved accuracy.

### 6.3. Loading the Model and Experiment Design

The Ultralytics library makes loading YOLOv8 models straightforward. In this project, we implement four distinct experimental scenarios to thoroughly evaluate model performance:

1. **Transfer Learning (Pre-trained)**: Loading a model pre-trained on COCO dataset and fine-tuning on our Pascal VOC dataset, using standard data augmentation.
   ```python
   model = YOLO("yolov8n.pt")  # Load pre-trained weights
   ```

2. **Transfer Learning with Enhanced Augmentation**: Using pre-trained weights but applying stronger data augmentation techniques during training, including mosaic, mixup, and geometric transformations.
   ```python
   model = YOLO("yolov8n.pt")  # Load pre-trained weights
   # Apply enhanced augmentation during training
   ```

3. **Training from Scratch**: Training a model with the same architecture but with randomly initialized weights, allowing it to learn solely from our dataset.
   ```python
   model = YOLO("yolov8n.yaml")  # Load only the model configuration
   ```

4. **Training from Scratch with Enhanced Augmentation**: Similar to approach #3, but with stronger data augmentation techniques to potentially improve generalization with limited data.
   ```python
   model = YOLO("yolov8n.yaml")  # Load only the model configuration
   # Apply enhanced augmentation during training
   ```

This comprehensive experimental design allows us to assess:
- The impact of transfer learning vs. learning from scratch
- The additional benefit of enhanced data augmentation
- The interaction between pre-training and augmentation strategies

Each model will be evaluated using the same metrics to enable direct comparisons across the four approaches.

In [4]:
# Cell 4: Model Training Function
def train_model(dataset_yaml, model_size='n', epochs=50, batch_size=16, img_size=640, 
                pretrained=True, project_name="yolo_training"):
    """
    Train a YOLOv8 model on a custom dataset
    
    Parameters:
    -----------
    dataset_yaml: str
        Path to the dataset YAML configuration file
    model_size: str
        Size of YOLOv8 model ('n', 's', 'm', 'l', 'x')
    epochs: int
        Number of training epochs
    batch_size: int
        Batch size for training
    img_size: int
        Image size for training
    pretrained: bool
        Whether to start with pretrained weights (recommended)
    project_name: str
        Name for the project folder to save results
    
    Returns:
    --------
    model: YOLO
        The trained YOLO model
    """
    print(f"Starting training with YOLOv8{model_size} for {epochs} epochs")
    
    # Initialize model
    if pretrained:
        model = YOLO(f"yolov8{model_size}.pt")  # load pretrained model
    else:
        model = YOLO(f"yolov8{model_size}.yaml")  # create new model from scratch
    
    # Log dataset yaml content
    print(f"Using dataset config: {dataset_yaml}")
    with open(dataset_yaml, 'r') as f:
        yaml_content = f.read()
        print(f"YAML Content:\n{yaml_content}")
    
    # Start training with absolute paths
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=project_name,
        name=f"yolov8{model_size}_custom",
        patience=10,  # early stopping patience
        save=True,  # save checkpoints
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=True
    )
    
    return model, results

In [5]:
# Cell 5: Model Validation Function
def validate_model(model, dataset_yaml):
    """
    Validate the trained model on validation dataset
    """
    # Run validation
    metrics = model.val(data=dataset_yaml, split='val', plots=True)
    
    # Print metrics
    print("\nValidation Metrics:")
    if metrics and hasattr(metrics, 'box'):
        print(f"mAP50-95 (Box): {metrics.box.map:.4f}")   # Mean Average Precision @ IoU=0.50:0.95
        print(f"mAP50 (Box):    {metrics.box.map50:.4f}") # Mean Average Precision @ IoU=0.50
        print(f"mAP75 (Box):    {metrics.box.map75:.4f}") # Mean Average Precision @ IoU=0.75
        print(f"Mean Precision (Box): {metrics.box.mp:.4f}") # Mean Precision across all classes
        print(f"Mean Recall (Box):    {metrics.box.mr:.4f}") # Mean Recall across all classes
        
        # For individual class metrics
        print("\nPer-class metrics (mAP50):")
        if hasattr(metrics.box, 'ap50'):
            class_names = model.names
            for i, ap in enumerate(metrics.box.ap50):
                if i < len(class_names):
                    print(f"  {class_names[i]}: {ap:.4f}")
                    
        print(f"\nValidation plots and results saved to: {metrics.save_dir}")
    else:
        print("Validation metrics or box metrics not available.")
        
    return metrics

In [6]:
# Cell 6: Model Export and Results Plotting
def export_model(model, format='onnx'):
    """
    Export trained model to different formats for deployment
    Supported formats: onnx, openvino, torchscript, tflite, etc.
    """
    # Export the model
    model.export(format=format)
    print(f"Model exported to {format} format")

def plot_training_results(results_path):
    """
    Plot training results
    """
    from ultralytics.utils.plotting import plot_results
    
    # Find the results.csv file
    if os.path.isdir(results_path):
        results_file = Path(results_path) / "results.csv"
    else:
        results_file = results_path
    
    # Plot results
    fig, ax = plot_results(results_file)
    plt.savefig(Path(results_path) / "training_results.png")
    plt.show()

## 7. Methods (5%)

This section outlines the methodologies employed for training and evaluating our YOLOv8 object detection model on the prepared Pascal VOC 2007 dataset.

### 7.1. Training Methodology

The core of our method involves training the YOLOv8 model using the Ultralytics framework. We explore four distinct training strategies, creating a 2×2 experimental design:

1. **Transfer Learning (Fine-tuning):**
   * **Initialization:** We start with a YOLOv8 model (e.g., `yolov8n.pt`) that has been pre-trained on a large-scale dataset like COCO.
   * **Training Process:** The pre-trained model is then fine-tuned on our Pascal VOC 2007 dataset using standard augmentation.
   * **Advantages:** Typically leads to faster convergence and better performance, especially with smaller custom datasets.

2. **Transfer Learning with Enhanced Augmentation:**
   * **Initialization:** Same as above, starting with COCO pre-trained weights.
   * **Training Process:** We apply stronger data augmentation during training, including increased mosaic, mixup, geometric transformations, etc.
   * **Hypothesis:** Enhanced augmentation may further improve performance by creating more diverse training samples.

3. **Training from Scratch:**
   * **Initialization:** We initialize a YOLOv8 model (via `yolov8n.yaml`) with random weights. The model architecture is defined, but it has no prior knowledge.
   * **Training Process:** The model is trained solely on our Pascal VOC 2007 dataset with standard augmentation.
   * **Purpose:** To understand the model's learning capacity on the specific dataset without influence from other data sources.

4. **Training from Scratch with Enhanced Augmentation:**
   * **Initialization:** Same as above, starting with random weights.
   * **Training Process:** We apply the same enhanced augmentation as in strategy #2.
   * **Hypothesis:** This may partially compensate for lack of pre-training by artificially increasing training data diversity.

This 2×2 experimental design allows us to isolate and measure:
- The effect of transfer learning vs. learning from scratch
- The impact of enhanced data augmentation
- The interaction between these two factors (whether augmentation benefits pre-trained models differently than models trained from scratch)

### 7.2. Evaluation Methodology

Model performance is assessed using standard object detection metrics, primarily calculated on the validation set. The Ultralytics framework automatically computes these after training if `val=True` (default) or when the `.val()` method is explicitly called.

**Key Evaluation Metrics:**
*   **Intersection over Union (IoU):** A measure of the overlap between a predicted bounding box and a ground-truth bounding box.
    \[ IoU = \frac{\text{Area of Overlap}}{\text{Area of Union}} \]
*   **Precision:** The accuracy of positive predictions.
    \[ Precision = \frac{\text{True Positives (TP)}}{\text{True Positives (TP) + False Positives (FP)}} \]
    (TP: Correctly detected object; FP: Incorrectly detected object, or detection where no object exists)
*   **Recall:** The ability of the model to find all relevant instances.
    \[ Recall = \frac{\text{True Positives (TP)}}{\text{True Positives (TP) + False Negatives (FN)}} \]
    (FN: A ground-truth object that the model failed to detect)
*   **Average Precision (AP):** The area under the Precision-Recall curve for a specific class. It summarizes the model's performance for that class across all recall levels.
*   **mean Average Precision (mAP):** The average of the AP values across all object classes. This is the primary metric for comparing object detection models.
    *   **mAP@0.5 (or mAP50):** mAP calculated at a single IoU threshold of 0.5. This means a detection is considered a True Positive if its IoU with the ground truth is >= 0.5.
    *   **mAP@0.5:0.95 (or mAP):** The average mAP over multiple IoU thresholds, typically from 0.5 to 0.95 with a step of 0.05 (COCO standard). This provides a more comprehensive evaluation across different levels of localization accuracy.

Our `validate_model` function in `train_yolov8_model.py` is used to trigger the validation process and print these metrics.

### 7.3. Software and Hardware
*   **Primary Library:** Ultralytics YOLOv8
*   **Deep Learning Framework:** PyTorch
*   **Key Python Packages:** `torch`, `torchvision`, `ultralytics`, `pyyaml`, `matplotlib`, `numpy`, `Pillow`, `tqdm`
*   **Development Environment:** Python 3.10, Jupyter Notebook
*   **Hardware:** Training was performed using a system equipped with an NVIDIA GeForce RTX 4090 Laptop GPU. CPU was used for some preprocessing tasks.

## 8. Experiments and Results (10%)

This section details the experiments conducted using our four-pronged approach to YOLOv8 model training on Pascal VOC 2007. We will report performance metrics and visualize results across all four experiments.

**Setup:**
* **Dataset:** Processed Pascal VOC 2007 (converted to YOLO format)
* **Model:** YOLOv8n (nano version)
* **Hardware:** NVIDIA GeForce RTX 4090 Laptop GPU
* **Common Parameters:**
  * Image Size (`imgsz`): 640
  * Batch Size: 16
  * Epochs: 50 for all experiments
  * Early stopping patience: 10 epochs

### 8.1. Experiment 1: Transfer Learning with Standard Augmentation

In this experiment, we fine-tune a YOLOv8n model pre-trained on the COCO dataset using our Pascal VOC 2007 data with standard data augmentation parameters.

**Parameters:**
* Model Initialization: `yolov8n.pt` (pre-trained weights)
* Augmentation: Default Ultralytics settings

### 8.2. Experiment 2: Transfer Learning with Enhanced Augmentation

Building on Experiment 1, we use the same pre-trained weights but apply stronger data augmentation techniques during training.

**Parameters:**
* Model Initialization: `yolov8n.pt` (pre-trained weights)
* Enhanced Augmentation:
  * Mosaic: 1.0 (probability)
  * Mixup: 0.5 (probability)
  * Rotation: ±20.0 degrees
  * Translation: 0.2
  * Shear: 10.0
  * And other enhancements (see code)

### 8.3. Experiment 3: Training from Scratch with Standard Augmentation

In this experiment, we train a YOLOv8n model with randomly initialized weights using only our Pascal VOC 2007 data with standard augmentation.

**Parameters:**
* Model Initialization: `yolov8n.yaml` (architecture only, random weights)
* Augmentation: Default Ultralytics settings

### 8.4. Experiment 4: Training from Scratch with Enhanced Augmentation

In our final experiment, we combine training from scratch with the enhanced augmentation techniques used in Experiment 2.

**Parameters:**
* Model Initialization: `yolov8n.yaml` (architecture only, random weights)
* Enhanced Augmentation: Same as Experiment 2

In [7]:
# Cell 7: Setup Paths and Parameters for All Experiments

# Set paths for your directory structure using absolute paths
workspace_dir = Path(os.path.dirname(os.path.abspath("__file__")))  # Modified for notebook
voc_path = workspace_dir / "datasets" / "VOCdevkit" / "VOC2007"
yolo_dataset_path = workspace_dir / "datasets" / "voc_yolo_converted"

# Check if VOC dataset exists
if not os.path.exists(voc_path):
    print(f"VOC dataset not found at {voc_path}")
else:
    # Set common training parameters for all experiments
    model_size = 'n'  # nano model (options: 'n', 's', 'm', 'l', 'x')
    epochs = 50
    batch_size = 16
    img_size = 640
    project_name = "yolo_training"
    
    # Define experiment variations
    transfer_learning = True        # For experiments 1 & 2
    enhanced_augmentation = False   # For experiments 2 & 4
    
    print("Parameters set for all experiments. Ready to begin processing.")

Parameters set for all experiments. Ready to begin processing.


In [8]:
# Cell 8: Define Training Functions with Enhanced Augmentation

# Function for transfer learning with enhanced augmentation
def train_model_augmented(dataset_yaml, model_size='n', epochs=50, batch_size=16, img_size=640, 
                         project_name="yolo_training", pretrained=True):
    """
    Train a YOLOv8 model with enhanced data augmentation
    
    Parameters:
    -----------
    pretrained: bool
        Whether to use transfer learning (True) or train from scratch (False)
    """
    train_type = "transfer learning" if pretrained else "scratch training"
    print(f"Starting {train_type} with YOLOv8{model_size} with enhanced augmentation for {epochs} epochs")
    
    # Initialize model based on whether we're doing transfer learning or from-scratch
    if pretrained:
        model = YOLO(f"yolov8{model_size}.pt")  # Load pretrained model
    else:
        model = YOLO(f"yolov8{model_size}.yaml")  # Create model from scratch
    
    # Name for the experiment
    exp_name = f"yolov8{model_size}_{'pretrained' if pretrained else 'scratch'}_augmented"
    
    # Start training with enhanced augmentation
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=project_name,
        name=exp_name,
        patience=10,
        save=True,
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=True,
        # Enhanced augmentation parameters
        mosaic=1.0,           # Increase mosaic probability (default is 1.0)
        mixup=0.5,            # Add mixup augmentation (default is 0.0)
        degrees=20.0,         # Rotation augmentation (default is 0.0)
        translate=0.2,        # Translation augmentation (default is 0.1)
        scale=0.2,            # Scale augmentation (default is 0.5)
        shear=10.0,           # Shear augmentation (default is 0.0)
        perspective=0.001,    # Perspective augmentation (default is 0.0)
        flipud=0.1,           # Vertical flip (default is 0.0)
        fliplr=0.5,           # Horizontal flip (default is 0.5)
        hsv_h=0.015,          # HSV-Hue augmentation (default is 0.015)
        hsv_s=0.7,            # HSV-Saturation augmentation (default is 0.7)
        hsv_v=0.4,            # HSV-Value augmentation (default is 0.4)
        copy_paste=0.1        # Copy-paste augmentation (default is 0.0)
    )
    
    return model, results

In [9]:
# Cell 9: Data Conversion (Run this cell to convert VOC to YOLO format)
# This cell may take some time to execute
converted_dataset_path = convert_voc_to_yolo(voc_path, yolo_dataset_path)
dataset_yaml = create_dataset_yaml(converted_dataset_path, name="voc_yolo")

Converting VOC dataset from /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/VOCdevkit/VOC2007 to YOLO format at /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted
Output directory /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted already exists. Removing existing data...
Found 5011 training images and 4952 validation images


Converting annotations: 100%|██████████| 4952/4952 [05:02<00:00, 16.36it/s]

Conversion completed!
Created dataset config at /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/voc_yolo.yaml


In [21]:
# Cell 10: Experiment 1 - Transfer Learning (Standard)
print("\n=== EXPERIMENT 1: TRANSFER LEARNING (STANDARD) ===\n")

model_tl, results_tl = train_model(
    dataset_yaml=dataset_yaml,
    model_size=model_size,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    pretrained=True,  # Use transfer learning
    project_name=project_name
)

# Get the results directory
results_dir_tl = results_tl.save_dir
print(f"Transfer learning results saved to {results_dir_tl}")

# Validate the model
print("\nValidating transfer learning model...")
metrics_tl = validate_model(model_tl, dataset_yaml)


=== EXPERIMENT 1: TRANSFER LEARNING (STANDARD) ===

Starting training with YOLOv8n for 50 epochs
Using dataset config: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/voc_yolo.yaml
YAML Content:
path: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted
train: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/images
val: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/images
test: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/images
names:
  0: aeroplane
  1: bicycle
  2: bird
  3: boat
  4: bottle
  5: bus
  6: car
  7: cat
  8: chair
  9: cow
  10: diningtable
  11: dog
  12: horse
  13: motorbike
  14: person
  15: pottedplant
  16: sheep
  17: sofa
  18: train
  19: tvmonitor

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Lapt

train: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:09<00:00, 511.29it/s]


train: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels.cache
val: Fast image access ✅ (ping: 3.5±0.8 ms, read: 16.9±5.7 MB/s, size: 86.9 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:08<00:00, 577.04it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache
Plotting labels to yolo_training/yolov8n_custom3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_training/yolov8n_custom3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4952      14976      0.783      0.669      0.743      0.523

50 epochs completed in 1.032 hours.
Optimizer stripped from yolo_training/yolov8n_custom3/weights/last.pt, 6.2MB
Optimizer stripped from yolo_training/yolov8n_custom3/weights/best.pt, 6.2MB

Validating yolo_training/yolov8n_custom3/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:41<00:00,  3.73it/s]


                   all       4952      14976      0.788      0.667      0.744      0.523
             aeroplane        205        311      0.816      0.726      0.811      0.558
               bicycle        250        389      0.826      0.725      0.807      0.589
                  bird        289        576      0.757      0.631      0.705      0.447
                  boat        176        393      0.675      0.552      0.615      0.357
                bottle        240        657      0.778      0.461      0.555      0.358
                   bus        183        254      0.787      0.715      0.797       0.66
                   car        775       1541      0.877       0.77      0.858      0.636
                   cat        332        370      0.809      0.735      0.803      0.598
                 chair        545       1374      0.749      0.463      0.574      0.366
                   cow        127        329      0.781      0.751      0.806      0.569
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:46<00:00,  6.62it/s]


                   all       4952      14976      0.793      0.663      0.745      0.524
             aeroplane        205        311      0.815       0.72      0.811      0.561
               bicycle        250        389      0.826      0.721      0.808      0.588
                  bird        289        576      0.763      0.627      0.705      0.447
                  boat        176        393       0.68      0.547      0.614      0.357
                bottle        240        657      0.794       0.46      0.557      0.361
                   bus        183        254      0.787      0.713      0.795       0.66
                   car        775       1541      0.877      0.767      0.859      0.636
                   cat        332        370      0.823      0.732      0.807      0.601
                 chair        545       1374      0.755      0.456      0.574      0.367
                   cow        127        329      0.789      0.748      0.806      0.569
           diningtabl

In [12]:
# Cell 11: Experiment 2 - Transfer Learning with Enhanced Augmentation
print("\n=== EXPERIMENT 2: TRANSFER LEARNING WITH ENHANCED AUGMENTATION ===\n")

model_tl_aug, results_tl_aug = train_model_augmented(
    dataset_yaml=dataset_yaml,
    model_size=model_size,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    pretrained=True,  # Use transfer learning
    project_name=project_name
)

# Get the results directory
results_dir_tl_aug = results_tl_aug.save_dir
print(f"Transfer learning with augmentation results saved to {results_dir_tl_aug}")

# Validate the model
print("\nValidating transfer learning model with augmentation...")
metrics_tl_aug = validate_model(model_tl_aug, dataset_yaml)


=== EXPERIMENT 2: TRANSFER LEARNING WITH ENHANCED AUGMENTATION ===

Starting transfer learning with YOLOv8n with enhanced augmentation for 50 epochs
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/voc_yolo.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mi

train: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:09<00:00, 549.81it/s]


train: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels.cache
val: Fast image access ✅ (ping: 2.8±0.4 ms, read: 16.5±5.0 MB/s, size: 86.9 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:08<00:00, 609.32it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache
Plotting labels to yolo_training/yolov8n_pretrained_augmented4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_training/yolov8n_pretrained_augmented4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4952      14976      0.763      0.646      0.718      0.459

50 epochs completed in 1.185 hours.
Optimizer stripped from yolo_training/yolov8n_pretrained_augmented4/weights/last.pt, 6.2MB
Optimizer stripped from yolo_training/yolov8n_pretrained_augmented4/weights/best.pt, 6.2MB

Validating yolo_training/yolov8n_pretrained_augmented4/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:48<00:00,  3.17it/s]


                   all       4952      14976      0.763      0.645      0.718      0.459
             aeroplane        205        311      0.834      0.713      0.803      0.478
               bicycle        250        389      0.834      0.736      0.805      0.539
                  bird        289        576      0.801       0.62      0.704      0.396
                  boat        176        393      0.673      0.471      0.535      0.287
                bottle        240        657      0.743      0.413       0.53      0.295
                   bus        183        254      0.806      0.688      0.775      0.586
                   car        775       1541      0.858       0.74      0.836      0.575
                   cat        332        370      0.753      0.765      0.805      0.537
                 chair        545       1374      0.727      0.399      0.524        0.3
                   cow        127        329      0.763      0.713      0.778      0.491
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/310 [00:00<?, ?it/s]

: 

In [13]:
#the above kernel crashed on validation so lets access the best weights and validate them
# Cell: Validate the model with the best weights
from ultralytics import YOLO

# Path to the saved weights
weights_path = "yolo_training/yolov8n_pretrained_augmented4/weights/best.pt"

# Load the model
model = YOLO(weights_path)

# Validate the model on the validation dataset
results = model.val(data="datasets/voc_yolo_converted/voc_yolo.yaml")

# Print validation results
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

# Print per-class metrics (mAP50)
print("\nPer-class metrics (mAP50):")
class_names = model.names
for i, ap in enumerate(results.box.ap50):
    print(f"  {class_names[i]}: {ap:.4f}")

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.7±0.8 ms, read: 21.2±5.1 MB/s, size: 87.4 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:09<00:00, 546.66it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:43<00:00,  7.18it/s]


                   all       4952      14976      0.763      0.645      0.717      0.459
             aeroplane        205        311      0.838      0.717      0.804      0.479
               bicycle        250        389      0.831      0.735      0.805      0.541
                  bird        289        576      0.801       0.62      0.703      0.396
                  boat        176        393      0.679      0.473      0.534      0.287
                bottle        240        657      0.739      0.409      0.529      0.294
                   bus        183        254      0.806      0.687      0.773      0.584
                   car        775       1541       0.86      0.741      0.836      0.575
                   cat        332        370      0.758      0.765      0.807       0.54
                 chair        545       1374      0.731      0.399      0.525        0.3
                   cow        127        329      0.763      0.714      0.778      0.491
           diningtabl

In [10]:
# Cell 12: Experiment 3 - Training from Scratch
print("\n=== EXPERIMENT 3: TRAINING FROM SCRATCH ===\n")

model_scratch, results_scratch = train_model(
    dataset_yaml=dataset_yaml,
    model_size=model_size,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    pretrained=False,  # Train from scratch
    project_name=project_name
)

# Get the results directory
results_dir_scratch = results_scratch.save_dir
print(f"From-scratch training results saved to {results_dir_scratch}")

# Validate the model
print("\nValidating from-scratch model...")
metrics_scratch = validate_model(model_scratch, dataset_yaml)


=== EXPERIMENT 3: TRAINING FROM SCRATCH ===

Starting training with YOLOv8n for 50 epochs
Using dataset config: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/voc_yolo.yaml
YAML Content:
path: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted
train: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/images
val: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/images
test: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/images
names:
  0: aeroplane
  1: bicycle
  2: bird
  3: boat
  4: bottle
  5: bus
  6: car
  7: cat
  8: chair
  9: cow
  10: diningtable
  11: dog
  12: horse
  13: motorbike
  14: person
  15: pottedplant
  16: sheep
  17: sofa
  18: train
  19: tvmonitor

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU,

train: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:09<00:00, 551.70it/s]


train: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels.cache
val: Fast image access ✅ (ping: 2.5±0.2 ms, read: 16.5±5.1 MB/s, size: 86.9 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:08<00:00, 550.97it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache
Plotting labels to yolo_training/yolov8n_custom5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_training/yolov8n_custom5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4952      14976      0.479      0.391      0.394      0.234

50 epochs completed in 1.100 hours.
Optimizer stripped from yolo_training/yolov8n_custom5/weights/last.pt, 6.2MB
Optimizer stripped from yolo_training/yolov8n_custom5/weights/best.pt, 6.2MB

Validating yolo_training/yolov8n_custom5/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
YOLOv8n summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:48<00:00,  3.21it/s]


                   all       4952      14976      0.478      0.392      0.394      0.234
             aeroplane        205        311      0.536      0.489      0.473      0.279
               bicycle        250        389      0.533      0.507      0.522      0.321
                  bird        289        576       0.39      0.226      0.212      0.109
                  boat        176        393       0.47      0.186      0.219      0.112
                bottle        240        657      0.332       0.07     0.0844     0.0384
                   bus        183        254      0.565      0.508      0.511      0.378
                   car        775       1541      0.663      0.538        0.6      0.415
                   cat        332        370       0.43      0.432       0.41      0.245
                 chair        545       1374      0.442      0.207      0.234      0.114
                   cow        127        329      0.395      0.432      0.345      0.207
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/310 [00:00<?, ?it/s]

: 

In [10]:
#the above kernel crashed on validation so lets access the best weights and validate them
# Cell: Validate the model with the best weights
from ultralytics import YOLO

# Path to the saved weights
weights_path = "yolo_training/yolov8n_custom5/weights/best.pt"

# Load the model
model = YOLO(weights_path)

# Validate the model on the validation dataset
results = model.val(data="datasets/voc_yolo_converted/voc_yolo.yaml")

# Print validation results
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

# Print per-class metrics (mAP50)
print("\nPer-class metrics (mAP50):")
class_names = model.names
for i, ap in enumerate(results.box.ap50):
    print(f"  {class_names[i]}: {ap:.4f}")

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
YOLOv8n summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.8±0.6 ms, read: 20.3±5.4 MB/s, size: 102.4 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:09<00:00, 518.08it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:42<00:00,  7.25it/s]


                   all       4952      14976      0.478      0.389      0.394      0.234
             aeroplane        205        311      0.539      0.486      0.472      0.279
               bicycle        250        389      0.529      0.506      0.524       0.32
                  bird        289        576      0.381      0.221      0.211      0.107
                  boat        176        393      0.485      0.186      0.219      0.112
                bottle        240        657      0.333       0.07     0.0841      0.038
                   bus        183        254      0.566      0.508      0.512      0.378
                   car        775       1541      0.663      0.537      0.601      0.415
                   cat        332        370      0.433      0.435      0.411      0.244
                 chair        545       1374      0.445      0.207      0.234      0.114
                   cow        127        329      0.391      0.426      0.347      0.207
           diningtabl

In [10]:
# Cell 13: Experiment 4 - Training from Scratch with Enhanced Augmentation
print("\n=== EXPERIMENT 4: TRAINING FROM SCRATCH WITH ENHANCED AUGMENTATION ===\n")

model_scratch_aug, results_scratch_aug = train_model_augmented(
    dataset_yaml=dataset_yaml,
    model_size=model_size,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    pretrained=False,  # Train from scratch
    project_name=project_name
)

# Get the results directory
results_dir_scratch_aug = results_scratch_aug.save_dir
print(f"From-scratch training with augmentation results saved to {results_dir_scratch_aug}")

# Validate the model
print("\nValidating from-scratch model with augmentation...")
metrics_scratch_aug = validate_model(model_scratch_aug, dataset_yaml)


=== EXPERIMENT 4: TRAINING FROM SCRATCH WITH ENHANCED AUGMENTATION ===

Starting scratch training with YOLOv8n with enhanced augmentation for 50 epochs
New https://pypi.org/project/ultralytics/8.3.131 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/voc_yolo.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=

train: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels... 5011 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5011/5011 [00:10<00:00, 484.53it/s]


train: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/train/labels.cache
val: Fast image access ✅ (ping: 3.0±1.5 ms, read: 16.7±4.8 MB/s, size: 86.9 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:09<00:00, 518.59it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache
Plotting labels to yolo_training/yolov8n_scratch_augmented/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_training/yolov8n_scratch_augmented
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4952      14976      0.379      0.292      0.268      0.129

50 epochs completed in 1.201 hours.
Optimizer stripped from yolo_training/yolov8n_scratch_augmented/weights/last.pt, 6.2MB
Optimizer stripped from yolo_training/yolov8n_scratch_augmented/weights/best.pt, 6.2MB

Validating yolo_training/yolov8n_scratch_augmented/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
YOLOv8n summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:41<00:00,  3.75it/s]


                   all       4952      14976      0.379      0.292      0.268      0.129
             aeroplane        205        311      0.446      0.363      0.373       0.15
               bicycle        250        389      0.354      0.401      0.374      0.191
                  bird        289        576      0.305     0.0868      0.107     0.0458
                  boat        176        393      0.336     0.0433     0.0755      0.032
                bottle        240        657      0.327    0.00304     0.0259    0.00965
                   bus        183        254      0.343      0.398      0.317      0.163
                   car        775       1541      0.513      0.464      0.486      0.261
                   cat        332        370      0.355      0.427      0.329      0.162
                 chair        545       1374      0.367      0.102       0.12     0.0492
                   cow        127        329      0.285      0.325      0.199      0.098
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]


: 

In [10]:


#the above kernel crashed on validation so lets access the best weights and validate them
# Cell: Validate the model with the best weights
from ultralytics import YOLO

# Path to the saved weights
weights_path = "yolo_training/yolov8n_scratch_augmented/weights/best.pt"

# Load the model
model = YOLO(weights_path)

# Validate the model on the validation dataset
results = model.val(data="datasets/voc_yolo_converted/voc_yolo.yaml")

# Print validation results
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

# Print per-class metrics (mAP50)
print("\nPer-class metrics (mAP50):")
class_names = model.names
for i, ap in enumerate(results.box.ap50):
    print(f"  {class_names[i]}: {ap:.4f}")

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
YOLOv8n summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.8±1.0 ms, read: 20.9±2.9 MB/s, size: 88.5 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:09<00:00, 515.19it/s]


val: New cache created: /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:43<00:00,  7.15it/s]


                   all       4952      14976      0.378      0.292      0.268      0.129
             aeroplane        205        311      0.446      0.363      0.374      0.151
               bicycle        250        389      0.352      0.401      0.373      0.191
                  bird        289        576       0.31     0.0885      0.106     0.0454
                  boat        176        393      0.323     0.0433     0.0764      0.032
                bottle        240        657      0.329    0.00304     0.0259    0.00967
                   bus        183        254      0.341      0.398      0.317      0.163
                   car        775       1541      0.512      0.464      0.486      0.262
                   cat        332        370      0.355      0.424      0.329      0.162
                 chair        545       1374      0.369      0.102       0.12     0.0493
                   cow        127        329      0.283      0.322      0.198     0.0977
           diningtabl

In [ ]:
# # Cell 14: Results Comparison and Visualization
# print("\n=== COMPARING RESULTS ACROSS ALL EXPERIMENTS ===\n")

# # Create a comparison table of results
# results_table = {
#     "Experiment": ["Transfer Learning", "Transfer Learning w/ Aug", "From Scratch", "From Scratch w/ Aug"],
#     "mAP50-95": [metrics_tl.box.map, metrics_tl_aug.box.map, metrics_scratch.box.map, metrics_scratch_aug.box.map],
#     "mAP50": [metrics_tl.box.map50, metrics_tl_aug.box.map50, metrics_scratch.box.map50, metrics_scratch_aug.box.map50],
#     "Precision": [metrics_tl.box.mp, metrics_tl_aug.box.mp, metrics_scratch.box.mp, metrics_scratch_aug.box.mp],
#     "Recall": [metrics_tl.box.mr, metrics_tl_aug.box.mr, metrics_scratch.box.mr, metrics_scratch_aug.box.mr]
# }

# # Display the table
# import pandas as pd
# results_df = pd.DataFrame(results_table)
# print(results_df.to_string(index=False))

# # Plot training results for all experiments
# plt.figure(figsize=(15, 10))

# # Function to plot results with a specific experiment name
# def plot_exp_results(results_dir, exp_name, color):
#     results_file = Path(results_dir) / "results.csv"
#     if results_file.exists():
#         df = pd.read_csv(results_file)
#         plt.plot(df['epoch'], df['metrics/mAP50(B)'], label=f"{exp_name} - mAP50", linestyle='-', color=color)
#         plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label=f"{exp_name} - mAP50-95", linestyle='--', color=color)

# # Plot each experiment
# plot_exp_results(results_dir_tl, "Transfer Learning", "blue")
# plot_exp_results(results_dir_tl_aug, "Transfer Learning w/ Aug", "green")
# plot_exp_results(results_dir_scratch, "From Scratch", "red")
# plot_exp_results(results_dir_scratch_aug, "From Scratch w/ Aug", "purple")

# plt.xlabel("Epoch")
# plt.ylabel("mAP")
# plt.title("Training Progress Comparison")
# plt.legend()
# plt.grid(True)
# plt.savefig("experiment_comparison.png")
# plt.show()

# print("\nExperiment comparison complete!")

## comparison

In [1]:
# Cell 1: Setup
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO

# Project directories
project_dir = "yolo_training"
experiments = {
    "Transfer Learning": "yolov8n_custom3",  # Update these to match your actual folder names
    "Transfer Learning w/ Aug": "yolov8n_pretrained_augmented4",
    "From Scratch": "yolov8n_custom5",  # Update with your actual folder names
    "From Scratch w/ Aug": "yolov8n_scratch_augmented"  # Update with your actual folder names
}

In [2]:
# Cell 2: Function to load metrics from saved model weights
def get_metrics_from_saved_model(experiment_name, folder_name):
    model_path = os.path.join(project_dir, folder_name, "weights", "best.pt")
    
    if os.path.exists(model_path):
        print(f"Loading model from {model_path}")
        model = YOLO(model_path)
        
        # Validate the model
        results = model.val(data="datasets/voc_yolo_converted/voc_yolo.yaml")
        
        return {
            "Experiment": experiment_name,
            "mAP50-95": results.box.map,
            "mAP50": results.box.map50,
            "Precision": results.box.mp,
            "Recall": results.box.mr
        }
    else:
        print(f"Warning: Model not found at {model_path}")
        return None

In [3]:
# Cell 3: Collect results from each experiment
results_list = []

for exp_name, folder_name in experiments.items():
    metrics = get_metrics_from_saved_model(exp_name, folder_name)
    if metrics:
        results_list.append(metrics)

# Create comparison DataFrame
if results_list:
    results_df = pd.DataFrame(results_list)
    print("\n=== EXPERIMENTAL RESULTS COMPARISON ===\n")
    print(results_df.to_string(index=False))
else:
    print("No results found. Please check your experiment directory names.")

Loading model from yolo_training/yolov8n_custom3/weights/best.pt
Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.9±0.7 ms, read: 17.7±6.9 MB/s, size: 84.6 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:43<00:00,  7.05it/s]


                   all       4952      14976      0.793      0.663      0.745      0.524
             aeroplane        205        311      0.815       0.72      0.811      0.561
               bicycle        250        389      0.826      0.721      0.808      0.588
                  bird        289        576      0.763      0.627      0.705      0.447
                  boat        176        393       0.68      0.547      0.614      0.357
                bottle        240        657      0.794       0.46      0.557      0.361
                   bus        183        254      0.787      0.713      0.795       0.66
                   car        775       1541      0.877      0.767      0.859      0.636
                   cat        332        370      0.823      0.732      0.807      0.601
                 chair        545       1374      0.755      0.456      0.574      0.367
                   cow        127        329      0.789      0.748      0.806      0.569
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:41<00:00,  7.55it/s]


                   all       4952      14976      0.763      0.645      0.717      0.459
             aeroplane        205        311      0.838      0.717      0.804      0.479
               bicycle        250        389      0.831      0.735      0.805      0.541
                  bird        289        576      0.801       0.62      0.703      0.396
                  boat        176        393      0.679      0.473      0.534      0.287
                bottle        240        657      0.739      0.409      0.529      0.294
                   bus        183        254      0.806      0.687      0.773      0.584
                   car        775       1541       0.86      0.741      0.836      0.575
                   cat        332        370      0.758      0.765      0.807       0.54
                 chair        545       1374      0.731      0.399      0.525        0.3
                   cow        127        329      0.763      0.714      0.778      0.491
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:41<00:00,  7.49it/s]


                   all       4952      14976      0.478      0.389      0.394      0.234
             aeroplane        205        311      0.539      0.486      0.472      0.279
               bicycle        250        389      0.529      0.506      0.524       0.32
                  bird        289        576      0.381      0.221      0.211      0.107
                  boat        176        393      0.485      0.186      0.219      0.112
                bottle        240        657      0.333       0.07     0.0841      0.038
                   bus        183        254      0.566      0.508      0.512      0.378
                   car        775       1541      0.663      0.537      0.601      0.415
                   cat        332        370      0.433      0.435      0.411      0.244
                 chair        545       1374      0.445      0.207      0.234      0.114
                   cow        127        329      0.391      0.426      0.347      0.207
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:40<00:00,  7.62it/s]


                   all       4952      14976      0.378      0.292      0.268      0.129
             aeroplane        205        311      0.446      0.363      0.374      0.151
               bicycle        250        389      0.352      0.401      0.373      0.191
                  bird        289        576       0.31     0.0885      0.106     0.0454
                  boat        176        393      0.323     0.0433     0.0764      0.032
                bottle        240        657      0.329    0.00304     0.0259    0.00967
                   bus        183        254      0.341      0.398      0.317      0.163
                   car        775       1541      0.512      0.464      0.486      0.262
                   cat        332        370      0.355      0.424      0.329      0.162
                 chair        545       1374      0.369      0.102       0.12     0.0493
                   cow        127        329      0.283      0.322      0.198     0.0977
           diningtabl

In [5]:
# Cell 4: Load and plot training progress for each experiment
plt.figure(figsize=(15, 10))

colors = {
    "Transfer Learning": "blue",
    "Transfer Learning w/ Aug": "green",
    "From Scratch": "red",
    "From Scratch w/ Aug": "purple"
}

for exp_name, folder_name in experiments.items():
    results_file = Path(project_dir) / folder_name / "results.csv"
    if results_file.exists():
        print(f"Loading training progress for {exp_name} from {results_file}")
        df = pd.read_csv(results_file)
        plt.plot(df['epoch'], df['metrics/mAP50(B)'], label=f"{exp_name} - mAP50", linestyle='-', color=colors[exp_name])
        plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label=f"{exp_name} - mAP50-95", linestyle='--', color=colors[exp_name])
    else:
        print(f"Warning: Results CSV not found for {exp_name} at {results_file}")

plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Training Progress Comparison")
plt.legend()
plt.grid(True)
plt.savefig("experiment_comparison.png")
plt.show()

Loading training progress for Transfer Learning from yolo_training/yolov8n_custom3/results.csv
Loading training progress for Transfer Learning w/ Aug from yolo_training/yolov8n_pretrained_augmented4/results.csv
Loading training progress for From Scratch from yolo_training/yolov8n_custom5/results.csv
Loading training progress for From Scratch w/ Aug from yolo_training/yolov8n_scratch_augmented/results.csv


<Figure size 1500x1000 with 1 Axes>

In [7]:
# Cell 5: Compare class-wise performance for different experiments
def get_classwise_metrics(experiment_name, folder_name):
    model_path = os.path.join(project_dir, folder_name, "weights", "best.pt")
    
    if os.path.exists(model_path):
        model = YOLO(model_path)
        results = model.val(data="datasets/voc_yolo_converted/voc_yolo.yaml")
        
        class_metrics = {}
        for i, ap in enumerate(results.box.ap50):
            class_metrics[model.names[i]] = ap
            
        return pd.Series(class_metrics, name=experiment_name)
    else:
        print(f"Warning: Model not found at {model_path}")
        return None

# Get class-wise results for each available experiment
class_results = []
for exp_name, folder_name in experiments.items():
    class_metrics = get_classwise_metrics(exp_name, folder_name)
    if class_metrics is not None:
        class_results.append(class_metrics)

if class_results:
    # Combine into a DataFrame
    class_df = pd.concat(class_results, axis=1)
    
    # Display class-wise comparison
    print("\n=== CLASS-WISE PERFORMANCE COMPARISON (mAP50) ===\n")
    print(class_df.to_string())
    
    # Visualize class-wise performance
    plt.figure(figsize=(14, 10))
    class_df.plot(kind='bar', figsize=(15, 8))
    plt.title('Class-wise mAP50 Comparison')
    plt.xlabel('Class')
    plt.ylabel('mAP50')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("class_performance_comparison.png")
    plt.show()
else:
    print("No class-wise results available.")

Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.7±0.2 ms, read: 14.2±4.0 MB/s, size: 84.5 KB)


val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:45<00:00,  6.75it/s]


                   all       4952      14976      0.793      0.663      0.745      0.524
             aeroplane        205        311      0.815       0.72      0.811      0.561
               bicycle        250        389      0.826      0.721      0.808      0.588
                  bird        289        576      0.763      0.627      0.705      0.447
                  boat        176        393       0.68      0.547      0.614      0.357
                bottle        240        657      0.794       0.46      0.557      0.361
                   bus        183        254      0.787      0.713      0.795       0.66
                   car        775       1541      0.877      0.767      0.859      0.636
                   cat        332        370      0.823      0.732      0.807      0.601
                 chair        545       1374      0.755      0.456      0.574      0.367
                   cow        127        329      0.789      0.748      0.806      0.569
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:42<00:00,  7.22it/s]


                   all       4952      14976      0.763      0.645      0.717      0.459
             aeroplane        205        311      0.838      0.717      0.804      0.479
               bicycle        250        389      0.831      0.735      0.805      0.541
                  bird        289        576      0.801       0.62      0.703      0.396
                  boat        176        393      0.679      0.473      0.534      0.287
                bottle        240        657      0.739      0.409      0.529      0.294
                   bus        183        254      0.806      0.687      0.773      0.584
                   car        775       1541       0.86      0.741      0.836      0.575
                   cat        332        370      0.758      0.765      0.807       0.54
                 chair        545       1374      0.731      0.399      0.525        0.3
                   cow        127        329      0.763      0.714      0.778      0.491
           diningtabl

...

                   all       4952      14976      0.478      0.389      0.394      0.234
             aeroplane        205        311      0.539      0.486      0.472      0.279
               bicycle        250        389      0.529      0.506      0.524       0.32
                  bird        289        576      0.381      0.221      0.211      0.107
                  boat        176        393      0.485      0.186      0.219      0.112
                bottle        240        657      0.333       0.07     0.0841      0.038
                   bus        183        254      0.566      0.508      0.512      0.378
                   car        775       1541      0.663      0.537      0.601      0.415
                   cat        332        370      0.433      0.435      0.411      0.244
                 chair        545       1374      0.445      0.207      0.234      0.114
                   cow        127        329      0.391      0.426      0.347      0.207
           diningtabl

val: Scanning /mnt/c/Users/kdb/Documents/msds/springMod2/deepLearning/2d_obj/datasets/voc_yolo_converted/val/labels.cache... 4952 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 310/310 [00:42<00:00,  7.25it/s]


                   all       4952      14976      0.378      0.292      0.268      0.129
             aeroplane        205        311      0.446      0.363      0.374      0.151
               bicycle        250        389      0.352      0.401      0.373      0.191
                  bird        289        576       0.31     0.0885      0.106     0.0454
                  boat        176        393      0.323     0.0433     0.0764      0.032
                bottle        240        657      0.329    0.00304     0.0259    0.00967
                   bus        183        254      0.341      0.398      0.317      0.163
                   car        775       1541      0.512      0.464      0.486      0.262
                   cat        332        370      0.355      0.424      0.329      0.162
                 chair        545       1374      0.369      0.102       0.12     0.0493
                   cow        127        329      0.283      0.322      0.198     0.0977
           diningtabl

<Figure size 1400x1000 with 0 Axes>

<Figure size 1500x800 with 1 Axes>

## none of my visuals will show in this notebook; see vissub.ipynb .


## Discussion of Results

Based on the visualizations in vissub.ipynb, we can observe several key trends and insights across our four experimental approaches to YOLOv8 object detection on the Pascal VOC dataset.

### Performance Comparison Between Training Approaches

The training progress comparison clearly shows a distinct hierarchy in performance:

*   **Transfer Learning (standard)** consistently outperforms all other approaches, achieving the highest mAP50 of approximately 0.74 by the end of training.
*   **Transfer Learning with Augmentation** begins with lower performance but steadily improves, reaching approximately 0.72 mAP50 after 50 epochs.
*   **Training from Scratch** demonstrates significantly lower performance than both transfer learning approaches, reaching only about 0.40 mAP50 after 50 epochs.
*   **Training from Scratch with Augmentation** performs worst overall, struggling to reach 0.27 mAP50 after the same training period.

### Impact of Transfer Learning vs. Training from Scratch

The performance gap between transfer learning and from-scratch approaches is substantial, with transfer learning models achieving approximately twice the mAP50 scores. This clearly demonstrates the value of leveraging pre-trained weights for object detection tasks, especially when working with relatively small datasets like Pascal VOC.

### Effect of Augmentation

Interestingly, data augmentation appears to have a negative impact on performance across both initialization strategies within our 50-epoch training window. However, a critical observation from the training loss curves reveals a significant dip in augmented model class loss around epoch 40 across all augmented models. This sharp decrease in loss suggests that the augmented models may be reaching an inflection point where they begin to benefit from the additional training variability.

This pattern strongly indicates that augmented models would likely outperform their non-augmented counterparts if training were extended beyond 50 epochs. The augmentation initially makes the learning task more difficult, but may ultimately lead to more robust feature learning and better generalization once the model has had sufficient time to process the more diverse training examples.

The delayed convergence of augmented models matches established patterns in deep learning, where regularization techniques (which data augmentation effectively serves as) often slow initial learning but improve final performance. The observed loss pattern suggests we may have stopped training just as the augmented models were beginning to realize their full potential.

### PR Curves and Class Performance

The Precision-Recall curves reveal significant class-specific differences:

*   **Transfer Learning models** show strong performance across many classes, with particular strength in vehicles (car, train), animals (horse, cat), and people.
*   **From-scratch models** struggle with most classes, with particularly poor performance on smaller objects like bottles and potted plants.
*   Certain classes consistently underperform across all models, notably "bottle" (0.555 in the best model), "chair" (0.574), and "potted plant" (0.516).
*   The most reliably detected classes are "person" (0.855), "car" (0.858), and "horse" (0.883).

### F1 Curves

The F1 curves reinforce these findings, with transfer learning models operating at significantly higher confidence thresholds (optimum around 0.45) compared to from-scratch models (optimum around 0.22), indicating much greater certainty in predictions. The from-scratch models with augmentation show the lowest optimal confidence threshold at just 0.17, suggesting very low confidence in predictions.

### Training Loss Analysis

The loss curves reveal that transfer learning models begin with much lower loss values, indicating the advantage of starting with pre-trained weights. The class loss for from-scratch models with augmentation remains particularly high throughout training until around epoch 40, where we observe a notable decrease. This pattern is consistent across all augmented models and suggests that with extended training, these models might eventually match or surpass their non-augmented versions as they better incorporate the augmented training examples.




## 9. Conclusion

This project successfully demonstrated the implementation of a 2D object detection pipeline using the YOLOv8n model on the Pascal VOC 2007 dataset. We covered the entire workflow, from data selection and thorough preprocessing to a comprehensive experimental evaluation using a 2×2 factorial design.

**Key Achievements:**

*   Successfully preprocessed the Pascal VOC 2007 dataset into a YOLOv8-compatible format.
*   Designed and implemented a systematic approach to compare four training strategies:
    1.  Transfer learning (standard augmentation assumed, or specify if it was a distinct "standard" set)
    2.  Transfer learning with enhanced augmentation
    3.  Training from scratch (standard augmentation assumed)
    4.  Training from scratch with enhanced augmentation
*   Evaluated model performance using standard object detection metrics (mAP, Precision, Recall) across all experimental conditions.
*   Analyzed the interaction between initialization strategy (transfer learning vs. from scratch) and data augmentation approach.

**Summary of Findings:**

*   **Performance Comparison & Impact of Transfer Learning:** Transfer learning is crucial for effective object detection with limited data. The models initialized with pre-trained weights (transfer learning) significantly outperformed those trained from scratch, achieving mAP50 scores nearly twice as high (e.g., standard transfer learning at ~0.74 mAP50 vs. from-scratch at ~0.40 mAP50). This demonstrates that the features learned from larger datasets like COCO transfer effectively to new object detection tasks.
*   **Effect of Augmentation & Convergence Analysis:** Data augmentation shows promise but requires extended training. Within our 50-epoch window, augmentation appeared to negatively impact initial performance (e.g., transfer learning with augmentation at ~0.72 mAP50, from-scratch with augmentation at ~0.27 mAP50). However, a significant loss reduction around epoch 40, particularly class loss for augmented models, strongly suggests these models were on the cusp of accelerated improvement. This indicates that augmentation likely provides long-term benefits with sufficient training time as models learn from greater data variability. The initialization strategy (transfer learning vs. from scratch) had a greater impact than augmentation choices within this short training regime.
*   **Model Capabilities & Object Characteristics:** Object characteristics influence detection difficulty. Transfer learning models showed strong performance across many classes, especially vehicles, animals, and people. From-scratch models struggled more, particularly with smaller or variably appearing objects like bottles, potted plants, chairs, and sofas, which proved challenging across all models. Larger, distinct objects (person: ~0.855, car: ~0.858, horse: ~0.883 in best models) were more consistently detected.
*   **Confidence & F1 Scores:** F1 curves reinforced these findings, with transfer learning models operating at significantly higher optimal confidence thresholds (~0.45) compared to from-scratch models (~0.22), and even lower for from-scratch with augmentation (~0.17), indicating greater certainty in predictions from transfer learning.

**Limitations of the Project:**

*   **Model Scale:** We experimented only with YOLOv8n (nano), the smallest variant. Larger models would likely yield higher accuracy but require more computational resources.
*   **Dataset Size:** While VOC 2007 is a benchmark, it's modest in size compared to datasets like COCO, which may limit performance, especially for models trained from scratch.
*   **Hyperparameter Consistency:** To ensure fair comparison, we maintained consistent hyperparameters across experiments rather than optimizing each approach separately.
*   **Training Duration:** All models were trained for 50 epochs. This may have particularly disadvantaged the from-scratch and augmented models, which typically require longer training to converge and realize the benefits of regularization or learning from diverse examples.

**Future Work & Potential Enhancements:**

*   **Extended Training for Augmented Models:** Explore longer training periods (e.g., 100-300 epochs) specifically for models utilizing data augmentation to fully assess their potential, given the promising loss convergence patterns observed.
*   **Experiment with Larger YOLOv8 Models:** Test YOLOv8s or YOLOv8m to evaluate the accuracy-resource trade-off on this dataset.
*   **Hyperparameter Optimization:** Conduct targeted hyperparameter tuning for each of the four training strategies.
*   **Advanced Augmentation & Curriculum Learning:** Investigate more sophisticated augmentation techniques and explore curriculum learning where augmentation difficulty is gradually increased.
*   **Broader Dataset Evaluation:** Test the generalizability of these findings on other object detection datasets.
*   **Deployment Strategies:** Develop model quantization or pruning for the best-performing model for efficient deployment.

Overall, this project provided valuable insights into the interplay between transfer learning and data augmentation for object detection tasks. The factorial experimental design allowed us to systematically evaluate these factors. For practical applications on similar datasets, transfer learning should be strongly preferred. When incorporating augmentation, the training schedule should be extended to fully capitalize on its benefits.